### Imports

In [ ]:
## pip install keras
## pip install tensorflow
## py -m venv keras
## pip show tensorflow

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout,Flatten, Embedding

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

In [2]:
spam = pd.read_csv("spam.csv")
spam.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
## Preprocessamento (tratamento)
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(spam["Category"])
print(y)

[0 0 1 ... 0 0 0]


In [4]:
## Dividir Dados em Treino
mensagens = spam["Message"].values
X_train,X_test, y_train,y_test = train_test_split(mensagens,y,test_size = 0.3)


In [6]:
token = Tokenizer(num_words = 1000)
token.fit_on_texts(X_train)

X_train = token.texts_to_sequences(X_train)
X_test = token.texts_to_sequences(X_test)

In [11]:
print(X_train)

[[ 32  39 416 ...   0   0   0]
 [ 32 295 242 ...   0   0   0]
 [208 595  12 ...   0   0   0]
 ...
 [777  31   3 ...   0   0   0]
 [  1 549   6 ...   0   0   0]
 [185  39  21 ...   0   0   0]]
(3900, 500)


In [9]:
X_train = pad_sequences(X_train, padding = "post", maxlen=500)
X_test = pad_sequences(X_test, padding = "post", maxlen=500)

In [12]:
print(X_train)

[[ 32  39 416 ...   0   0   0]
 [ 32 295 242 ...   0   0   0]
 [208 595  12 ...   0   0   0]
 ...
 [777  31   3 ...   0   0   0]
 [  1 549   6 ...   0   0   0]
 [185  39  21 ...   0   0   0]]
(3900, 500)


In [13]:
## vacabulario
print(len(token.word_index))

7516


In [14]:
## Criando Modelo
modelo = Sequential()

modelo.add(Embedding(input_dim = len(token.word_index),output_dim=50,input_length=500))
modelo.add(Flatten())

modelo.add(Dense( units=10, activation="relu"))
modelo.add(Dropout(0.1))
modelo.add(Dense( units=1, activation="sigmoid"))

In [15]:
## Montar a estrutura do Modelo
modelo.compile(loss = "mean_squared_error", optimizer = "adam", metrics= ["accuracy"] )
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 50)           375800    
                                                                 
 flatten (Flatten)           (None, 25000)             0         
                                                                 
 dense (Dense)               (None, 10)                250010    
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 625,821
Trainable params: 625,821
Non-trainable params: 0
_________________________________________________________________


In [16]:
## Treinar o Modelo
modelo.fit(X_train, y_train,epochs=20,batch_size=10, verbose= True, validation_data=(X_test,y_test))

Epoch 1/20
390/390 [==============================] - 3s 7ms/step - loss: 0.0713 - accuracy: 0.9197 - val_loss: 0.0388 - val_accuracy: 0.9850
Epoch 2/20
390/390 [==============================] - 2s 6ms/step - loss: 0.0361 - accuracy: 0.9826 - val_loss: 0.0275 - val_accuracy: 0.9862
Epoch 3/20
390/390 [==============================] - 2s 6ms/step - loss: 0.0276 - accuracy: 0.9877 - val_loss: 0.0229 - val_accuracy: 0.9868
Epoch 4/20
390/390 [==============================] - 2s 6ms/step - loss: 0.0213 - accuracy: 0.9910 - val_loss: 0.0186 - val_accuracy: 0.9886
Epoch 5/20
390/390 [==============================] - 2s 6ms/step - loss: 0.0159 - accuracy: 0.9931 - val_loss: 0.0169 - val_accuracy: 0.9880
Epoch 6/20
390/390 [==============================] - 2s 6ms/step - loss: 0.0123 - accuracy: 0.9946 - val_loss: 0.0138 - val_accuracy: 0.9910
Epoch 7/20
390/390 [==============================] - 2s 6ms/step - loss: 0.0094 - accuracy: 0.9969 - val_loss: 0.0138 - val_accuracy: 0.9898
Epoch 

In [17]:
loss,accuracy = modelo.evaluate(X_test,y_test)
print("Loss:",loss)
print("Accuracy:",accuracy)

53/53 [==============================] - 0s 3ms/step - loss: 0.0093 - accuracy: 0.9904
Loss: 0.009257901459932327
Accuracy: 0.9904305934906006


In [18]:
nova_previsao = modelo.predict(X_test)
print(nova_previsao)

53/53 [==============================] - 0s 3ms/step
[[9.4431967e-01]
 [2.9497866e-07]
 [9.1592181e-01]
 ...
 [8.9277371e-08]
 [6.5048361e-10]
 [2.0291649e-19]]


In [19]:
prev = (nova_previsao > 0.5)
print(prev)

[[ True]
 [False]
 [ True]
 ...
 [False]
 [False]
 [False]]


In [20]:
cm = confusion_matrix(y_test, prev)
print(cm)

[[1459    3]
 [  13  197]]
